In [1]:
import pandas as pd
from tqdm import tqdm
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
def query_pair(contract_id):
    
    sample_transport = RequestsHTTPTransport(url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                                            verify=True, retries=3)
    
    client = Client(transport=sample_transport)
    
    query = gql(
        '{'
            f'pair(id: "{contract_id}")'
            '{'
                'token0 {'
                    'id\n'
                    'symbol\n'
                    'name\n'
                    'derivedETH\n'
                '}'
                'token1 {'
                    'id\n'
                    'symbol\n'
                    'name\n'
                    'derivedETH\n'
                '}'
                'reserve0\n'
                'reserve1\n'
                'reserveUSD\n'
                'txCount'
            '}'
        '}'
    )
    
    response = client.execute(query)
    
    token0 = response['pair']['token0']['symbol']
    token1 = response['pair']['token1']['symbol']
    txCount = response['pair']['txCount']
    reserve0 = response['pair']['reserve0']
    reserve1 = response['pair']['reserve1']
    reserveUSD = response['pair']['reserveUSD']
    
    return token0, token1, txCount, reserve0, reserve1, reserveUSD

In [4]:
def get_pool_v2_reserves_history(contract_id: str) -> list:
    """get information about reserves updates conform given contract ID from Uniswap V2
    Args:
        contract_id (str): hash-sum of contract reserves updates history of which it is required to get
    Returns:
        list: array of reserves updates, where each record is inner array
    """
    sample_transport = RequestsHTTPTransport(url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                                            verify=True, retries=3)
    
    client = Client(transport=sample_transport)
    
    daily_data = []
    last_date = 0

    for skip in tqdm(range(0, 2)):
        try:
            contract_id = contract_id
            query = gql(
                'query{\n'
                 'pairDayDatas(first: 1000, orderBy: date, orderDirection: asc,\n'
                   'where: {\n'
                     f'pairAddress: "{contract_id}",\n'
                     f'date_gt: {last_date}\n'
                   '}\n'
                 ') {\n'
                     'date\n'
                     'dailyVolumeToken0\n'
                     'dailyVolumeToken1\n'
                     'dailyVolumeUSD\n'
                     'reserveUSD\n'
                     'reserve0\n'
                     'reserve1\n'
                 '}\n'
                '}\n'
            )

            response = client.execute(query)
            last_date = response['pairDayDatas'][-1]['date']
            daily_data.extend(response['pairDayDatas'])

        except Exception as e:
            print(e)
            
    return daily_data

In [5]:
def get_pool_v2_history(contract_id: str, range_limit: int=100) -> list:
    """get transaction history for given contract id
    Args:
        contract_id (str): hash-sum of required contract
        range_limit (int, optional): how many data fragments required (one fragment has 1000 records). Defaults to 100.
    Returns:
        list: list of transactions history, where each transaction is an inner array
    """
    sample_transport = RequestsHTTPTransport(url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                                            verify=True, retries=3)

    all_swaps = []
    last_timestamp = int(datetime.datetime.now().timestamp()) # todo: get current timestamp

    client = Client(transport=sample_transport)

    max_retries = 10
    retries = 0
    for skip in tqdm(range(0, range_limit)):
        try:
            query = gql(
            'query {\n'
                f'swaps(first: 1000, where: {{ pair: "{contract_id}", timestamp_lt: {last_timestamp} }} orderBy: timestamp, orderDirection: desc) {{\n'
                'transaction {\n'
                    'id\n'
                    'timestamp\n'
                '}\n'
                'id\n'
                'pair {\n'
                    'token0 {\n'
                    'id\n'
                    'symbol\n'
                    '}\n'
                    'token1 {\n'
                    'id\n'
                    'symbol\n'
                    '}\n'
                '}\n'
                'amount0In\n'
                'amount0Out\n'
                'amount1In\n'
                'amount1Out\n'
                'amountUSD\n'
                'sender\n'
                'to\n'
                '}\n'
            '}\n')

            # get current response and extract from it last timestamp
            response = client.execute(query)
            last_timestamp = response['swaps'][-1]['transaction']['timestamp']
            
            # extend swaps history with current response
            all_swaps.extend(response['swaps'])

        except Exception as e:
            retries += 1
            
            if retries > max_retries:
                print(e)
                
                return all_swaps
        else:
            retries = 0

    return all_swaps

In [6]:
def get_pool_v2_mints(contract_id: str, range_limit: int) -> list:
    sample_transport = RequestsHTTPTransport(url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                                            verify=True, retries=3)

    client = Client(transport=sample_transport)

    all_mints = []
    last_timestamp = 1641279150 # todo: replace current timestamp

    for skip in tqdm(range(0, range_limit)):
        try:
            query = gql(
                'query {\n'
                'mints(first: 1000, \n'
                'where: { '
                    f'pair: "{contract_id}", \n'
                    f'timestamp_lt: {last_timestamp}\n'
                '}, \n'
                'orderBy: timestamp, \n'
                'orderDirection: desc\n'
                ') {\n'
                'transaction {\n'
                    'id\n'
                    'timestamp\n'
                '}\n'
                'to\n'
                'liquidity\n'
                'amount0\n'
                'amount1\n'
                'amountUSD\n'
                '}\n'
                '}\n'
            )

            response = client.execute(query)
            last_timestamp = response['mints'][-1]['transaction']['timestamp']
            all_mints.extend(response['mints'])

        except Exception as e:
            print(e)
            
    return all_mints

In [7]:
def get_pool_v2_burns(contract_id: str, range_limit: int) -> list:
    """get information about burns conform specified contract id
    Args:
        contract_id (str): contract id for which is required to find burns
        range_limit (int): how many fragments it is required to get (fragment contains 1000 records)
    Returns:
        list: array of arrays representing pool burns data
    """
    sample_transport = RequestsHTTPTransport(url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                                            verify=True, retries=3)

    client = Client(transport=sample_transport)
    
    all_burns = []
    last_timestamp = 1641279150 # todo: replace current timestamp

    for skip in tqdm(range(0, range_limit)):
        try:
            query = gql(
                'query {\n'
                'burns(first: 1000, \n'
                'where: { '
                    f'pair: "{contract_id}", \n'
                    f'timestamp_lt: {last_timestamp}\n'
                '}, \n'
                'orderBy: timestamp, \n'
                'orderDirection: desc\n'
                ') {\n'
                'transaction {\n'
                    'id\n'
                    'timestamp\n'
                '}\n'
                'to\n'
                'liquidity\n'
                'amount0\n'
                'amount1\n'
                'amountUSD\n'
                '}\n'
                '}\n'
            )

            response = client.execute(query)
            last_timestamp = response['burns'][-1]['transaction']['timestamp']
            all_burns.extend(response['burns'])

        except Exception as e:
            print(e)
            
    return all_burns

In [8]:
def list_to_reserves_dictionary(daily_reserve: list) -> dict:
    """transform one reserve update data array into dictionary
    Args:
        daily_reserve (list): one reserve update record in array format
    Returns:
        dict: dictionary of one reserve update record
    """
    # reserve information
    reserve0 = daily_reserve['reserve0']
    reserve1 = daily_reserve['reserve1']
    reserveUSD = daily_reserve['reserveUSD']
    
    # daily volume info
    dailyVolumeToken0 = daily_reserve['dailyVolumeToken0']
    dailyVolumeToken1 = daily_reserve['dailyVolumeToken1']
    
    # date of reserve info taken
    date = daily_reserve['date']
    
    return {
        'reserve0': reserve0,
        'reserve1': reserve1,
        'reserveUSD': reserveUSD,
        'dailyVolumeToken0': dailyVolumeToken0,
        'dailyVolumeToken1': dailyVolumeToken1,
        'date': date
    }

In [9]:
def list_to_transaction_dictionary(transaction: list) -> dict:
    """transform transaction data present in array form into dictionary
    Args:
        transaction (list): one transaction data in array format
    Returns:
        dict: dictionary of one transaction record data
    """
    if transaction['amount0In'] != '0':
        token_in = transaction['pair']['token0']['symbol']
        token_out = transaction['pair']['token1']['symbol']
        amount_in = transaction['amount0In']
        amount_out = transaction['amount1Out']
    else:
        token_in = transaction['pair']['token1']['symbol']
        token_out = transaction['pair']['token0']['symbol']
        amount_in = transaction['amount1In']
        amount_out = transaction['amount0Out']
        
    amount_usd = transaction['amountUSD']
    sender = transaction['sender']
    to = transaction['to']
    timestamp = transaction['transaction']['timestamp']
    txd = transaction['transaction']['id']
    
    return {
        'token_in': token_in,
        'token_out': token_out,
        'amount_in': amount_in,
        'amount_out': amount_out,
        'amount_usd': amount_usd,
        'timestamp': timestamp,
        'sender': sender,
        'to': to,
        'txd': txd
    }

In [10]:
def list_to_mints_dictionary(mint: list) -> dict:
    """transform mint record in form of array into dictionary
    Args:
        mint (list): mint record in form of array
    Returns:
        dict: dictionary representing mint record
    """
    amount0 = mint['amount0']
    amount1 = mint['amount1']
    amountUSD = mint['amountUSD']
    liquidity = mint['liquidity']
    timestamp = mint['transaction']['timestamp']
    
    return {
        'amount0': amount0,
        'amount1': amount1,
        'amountUSD': amountUSD,
        'liquidity': liquidity,
        'timestamp': timestamp,
    }

In [11]:
def pool_history_to_df(pool_history: list) -> pd.DataFrame:
    """transform array of arrays representing transaction history into pandas dataframe
    Args:
        pool_history (list): array of arrays representing transaction history
    Returns:
        pd.DataFrame: pandas dataframe of transaction history
    """
    # transform transactions list of lists into list of dictionaries
    all_swaps_transformed = [list_to_transaction_dictionary(swap) for swap in pool_history]

    # make a dataframe from list of dictionaries and fix data type for specific columns
    swaps_df = pd.DataFrame(all_swaps_transformed)
    swaps_df.timestamp = pd.to_datetime(swaps_df.timestamp, unit='s')
    swaps_df['amount_in'] = swaps_df['amount_in'].astype('float')
    swaps_df['amount_out'] = swaps_df['amount_out'].astype('float')
    swaps_df['amount_usd'] = swaps_df['amount_usd'].astype('float')

    return swaps_df

In [12]:
def pool_mints_to_df(mints_list: list) -> pd.DataFrame:
    """transform array of arrays representing mints into pandas dataframe
    Args:
        reserves_list (list): array of arrays representing mints
    Returns:
        pd.DataFrame: pandas dataframe of mints
    """
    
    # transform list of dictionaries into df
    all_mints_df = pd.DataFrame([list_to_mints_dictionary(mint) for mint in mints_list])
    
    all_mints_df['amount0'] = all_mints_df.amount0.astype('float')
    all_mints_df['amount1'] = all_mints_df.amount1.astype('float')
    all_mints_df['amountUSD'] = all_mints_df.amountUSD.astype('float')
    all_mints_df['liquidity'] = all_mints_df.liquidity.astype('float')
    all_mints_df['timestamp'] = pd.to_datetime(all_mints_df['timestamp'], unit='s')
    
    return all_mints_df

In [13]:
def pool_burns_to_df(burns_list: list) -> pd.DataFrame:
    """transform array of arrays representing burns into pandas dataframe
    Args:
        reserves_list (list): array of arrays representing burns
    Returns:
        pd.DataFrame: pandas dataframe of burns
    """
    
    # transform list of dictionaries into df
    # !!! IMPORTANT: for burns the same list to dictionary transformer can be used as for mints
    all_burns_df = pd.DataFrame([list_to_mints_dictionary(burn) for burn in burns_list])
    
    all_burns_df['amount0'] = all_burns_df.amount0.astype('float')
    all_burns_df['amount1'] = all_burns_df.amount1.astype('float')
    all_burns_df['amountUSD'] = all_burns_df.amountUSD.astype('float')
    all_burns_df['liquidity'] = all_burns_df.liquidity.astype('float')
    all_burns_df['timestamp'] = pd.to_datetime(all_burns_df['timestamp'], unit='s')
    
    return all_burns_df

In [14]:
def pool_reserves_to_df(reserves_list: list) -> pd.DataFrame:
    """transform array of arrays representing reserves data into pandas dataframe
    Args:
        reserves_list (list): array of arrays representing reserve updates data
    Returns:
        pd.DataFrame: pandas dataframe of reserves updates
    """
    
    # transform list of dictionaries into df
    daily_data_df = pd.DataFrame([list_to_reserves_dictionary(x) for x in reserves_list])
    
    daily_data_df['reserve0'] = daily_data_df['reserve0'].astype('float')
    daily_data_df['reserve1'] = daily_data_df['reserve1'].astype('float')
    daily_data_df['reserveUSD'] = daily_data_df['reserveUSD'].astype('float')
    daily_data_df['dailyVolumeToken1'] = daily_data_df['dailyVolumeToken1'].astype('float')
    daily_data_df['date'] =  pd.to_datetime(daily_data_df['date'], unit='s')
    
    return daily_data_df

In [15]:
def pool_mints_to_df(mints_list: list) -> pd.DataFrame:
    """transform array of arrays representing mints into pandas dataframe
    Args:
        reserves_list (list): array of arrays representing mints
    Returns:
        pd.DataFrame: pandas dataframe of mints
    """
    
    # transform list of dictionaries into df
    all_mints_df = pd.DataFrame([list_to_mints_dictionary(mint) for mint in mints_list])
    
    all_mints_df['amount0'] = all_mints_df.amount0.astype('float')
    all_mints_df['amount1'] = all_mints_df.amount1.astype('float')
    all_mints_df['amountUSD'] = all_mints_df.amountUSD.astype('float')
    all_mints_df['liquidity'] = all_mints_df.liquidity.astype('float')
    all_mints_df['timestamp'] = pd.to_datetime(all_mints_df['timestamp'], unit='s')
    
    return all_mints_df

In [16]:
def filter_swaps(all_swaps):
    direct_swaps = list(filter(lambda x: ((x['amount0In'] != '0') ^ (x['amount1In'] !='0')) & ((x['amount0Out'] != '0') ^ (x['amount1Out'] != '0')) & (not ((x['amount0In'] == '0') & (x['amount0Out'] == '0')))  &  (not ((x['amount1In'] == '0') & (x['amount1Out'] == '0'))), all_swaps))
    other_swaps = list(filter(lambda x: not (((x['amount0In'] != '0') ^ (x['amount1In'] !='0')) & ((x['amount0Out'] != '0') ^ (x['amount1Out'] != '0')) & (not ((x['amount0In'] == '0') & (x['amount0Out'] == '0')))  &  (not ((x['amount1In'] == '0') & (x['amount1Out'] == '0')))), all_swaps))
    
    return direct_swaps, other_swaps

In [17]:
stablecoins = {
    'USDT': '0xdac17f958d2ee523a2206206994597c13d831ec7',
    'USDC': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'UST':  '0xa47c8bf37f92abed4a126bda807a7b7498661acd',
    'DAI':  '0x6b175474e89094c44da98b954eedeac495271d0f',
    'BUSD': '0x4fabb145d64652a948d72533023f6e7a623c7c53',
    'TUSD': '0x0000000000085d4780b73119b644ae5ecd22b376',
}

stablecoins_ids = list(stablecoins.values())
stablecoins_symbols = list(stablecoins.keys())

In [18]:
import numpy as np
def get_stable_info(token0, token1):
    # preserve stablecoin order
    for stable_symbol in stablecoins_symbols:
        if token0 == stable_symbol:
            return 0, True, token1 == stable_symbol
        
        if token1 == stable_symbol:
            return 1, False, True

In [19]:
import json, os

def extract_history(contract_id):
    all_swaps = get_pool_v2_history(contract_id, 30)
    all_swaps, other_swaps = filter_swaps(all_swaps)
    swaps_df = pool_history_to_df(all_swaps)

    reserves = get_pool_v2_reserves_history(contract_id)
    
    try:
        reserves_df = pool_reserves_to_df(reserves)
    except Exception as e:
        print(e)
        
        return
    
    token0, token1, txCount, reserve0, reserve1, reserveUSD = query_pair(contract_id)
    stable_index, is_token0_stable, is_token1_stable = get_stable_info(token0, token1)
    
    BASE_PAIR_DATA_PATH = f"data/all_selected/{token0}_{token1}"
    os.makedirs(BASE_PAIR_DATA_PATH, exist_ok=True)  # succeeds even if directory exists.
    
    config = {
        'contract_id': contract_id,
        'token0': token0,
        'token1': token1,
        'txCount': txCount,
        'reserve0': reserve0,
        'reserve1': reserve1,
        'reserveUSD': reserveUSD,
        'stable_index': stable_index,
        'is_token0_stable': is_token0_stable,
        'is_token1_stable': is_token1_stable
    }
    
    with open(f'{BASE_PAIR_DATA_PATH}/config.json', 'w') as f:
        json.dump(config, f)

    swaps_df.to_pickle(f'{BASE_PAIR_DATA_PATH}/{token0}_{token1}_swaps.pkl')
    reserves_df.to_pickle(f'{BASE_PAIR_DATA_PATH}/{token0}_{token1}_reserves.pkl')

In [27]:
contracts_list = ['0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
 '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852',
 '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5',
 '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f',
 '0x61b62c5d56ccd158a38367ef2f539668a06356ab',
 '0x87da823b6fc8eb8575a235a824690fda94674c88',
 '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11',
 '0xf6dcdce0ac3001b2f67f750bc64ea5beb37b5824',
 '0x6591c4bcd6d7a1eb4e537da8b78676c1576ba244',
 '0xccab68f48531215b0707e8d908c43e7de73dbdbc',
 '0xb20bd5d04be54f870d5c0d3ca85d82b34b836405',
 '0xd6ef070951d008f1e6426ad9ca1c4fcf7220ee4d',
 '0x5233349957586a8207c52693a959483f9aeaa50c',
 '0x8c1c499b1796d7f3c2521ac37186b52de024e58c',
 '0x873056a02255872514f05249d93228d788fe4fb4',
 '0x700fc86c46299cf2a8fd86edadae3f57014351b0',
 '0xc50ef7861153c51d383d9a7d48e6c9467fb90c38',
 '0x4b70ccd1cf9905be1faed025eadbd3ab124efe9a',
 '0x6bd8ca9d141aa95842b41e1431a244c309c9008c',
 '0xeafad3065de347b910bb88f09a5abe580a09d655',
 '0xc99a74145682c4b4a6e9fa55d559eb49a6884f75',
 '0x34856be886a2dba5f7c38c4df7fd86869ab08040',
 '0x67b3825348521b94828127f1ee31da80ee67d285',
 '0xb022e08adc8ba2de6ba4fecb59c6d502f66e953b',
 '0xf6c4e4f339912541d3f8ed99dba64a1372af5e5b',
 '0xfb1c171874d77627ca7c40dc9f8dd69fd2b0b5d0',
 '0x860425be6ad1345dc7a3e287facbf32b18bc4fae',
 '0xdcec563a15b0de0fdfc19c97d5ee18a5570fed7f',
 '0xf8d99cf7046dedcb1dc8cfc309aa96946c9b9db2',
 '0x676ce85f66adb8d7b8323aeefe17087a3b8cb363',
 '0x6f118ecebc31a5ffe49b87c47ea80f93a2af0a8a',
 '0x0ae8cb1f57e3b1b7f4f5048743710084aa69e796',
 '0x9f9373b2b5fe5f6b32f171191946b0be30e0bd30',
 '0x9e3b47b861b451879d43bba404c35bdfb99f0a6c',
 '0xa626eb9cc7dec00703586414d0811e1ba2021443',
 '0x21c5918ccb42d20a2368bdca8feda0399ebfd2f6',
 '0xf3b675df63fb4889180d290a338fc15c0766fd64',
 '0x4cd36d6f32586177e36179a810595a33163a20bf',
 '0xfcd13ea0b906f2f87229650b8d93a51b2e839ebd',
 '0x0e9c8107682ab88604b4fbf847eeeceacf38e9e6',
 '0xc88ac988a655b91b70def427c8778b4d43f2048d',
 '0xa5f0cf205af1f5b02c00ba7ab834824c01855b54',
 '0x5b1e45ca08fa4d65aa7fdcf9e116990fb7fce73b',
 '0x684b00a5773679f88598a19976fbeb25a68e9a5f',
 '0xa2f6a219a51b4682e34a13a94c160d6c79cdca35',
 '0x181655fee818e1e22c8aebb780c716e15b6f29aa',
 '0xd75d1b30967d94b105f82f572ae7591cc3c48beb',
 '0x85673c92f0f27a9c4d8c221f6bfefa33b716338a',
 '0x1cd926f3e12f7b6c2833fbe7277ac53d529a794e',
 '0x7b28470032da06051f2e620531adbaeadb285408',
 '0x9bd82673c50acb4a3b883d61e070a3c8d9b08e10',
 '0x8672aa02f8a5b6eb6bb0915ce6c65f771d705127',
 '0xdfa42ba0130425b21a1568507b084cc246fb0c8f',
 '0xd372a3221021df72eda38f77117d3a95f057e163',
 '0xd3d8c734f06229e36febd07505d8f57b7b78af7c',
 '0x1e45eae7461c56529e5cc335f6b1f797576f8a27',
 '0x22527f92f43dc8bea6387ce40b87ebaa21f51df3',
 '0xa40bb1c47f6dd27142a2bd7c93bfa98db9d1f5c5',
 '0x2dc9d00da9542f91d5391be53ed1d58a04ec1c07',
 '0x01388f9242964e2aaadef6379eb92276acb5520e',
 '0x803fc05552868c40336482dfdda36c2e13e6d5d6',
 '0x5340c7b63cef225e7c5e4620d4ff6c57ee585dd1',
 '0xd876bea7f5121a8e21459224e58aec6c933a16e9',
 '0x0e20642b32567f2dc74f149663fc474f534e1d5a',
 '0x0b41854f5d251c12b1de6a88dd4292944f04305c',
 '0x05a4fa8d7672bafc097223ce1188fa8940db2ddf',
 '0xbbc95e1eb6ee476e9cbb8112435e14b372563038',
 '0x1c19fd9bd77d4383c66d9a6ecf1bc67d462c477f',
 '0xe53533f78787c63735c77c9e2f9c60081b942cdb',
 '0x169bf778a5eadab0209c0524ea5ce8e7a616e33b',
 '0xaaa2bb0212ec7190dc7142cd730173b0a788ec31',
 '0x98f90bfc702ec548d21b5a566a4df6853e2890d4',
 '0xc4fa9fd2e51e41c2be2c3a77c53710b224e1b31d',
 '0x3155acd9f75915fcc21d34035f440da7040bd3ba',
 '0xa39d7a85553a46faeb3ba5e0c49d6a5db67df30f',
 '0x82cd7e4b567ad7d2a00c3cc93ac968ccb61cafca',
 '0x91b1b853c1426c4aa78cac984c6f6dd1e80b0c4f',
 '0xe93dc496dbc669d7ee4f03b0eb0a10bb13a4b2a4',
 '0x73c2c52ffe660bf17b9dfd1a170bb4e3ebf48d9d',
 '0x6b5fc4a09ecfa187c4e5b26a52636d2495f78692',
 '0xb8b7c440c36e31686bf1e1bdca76a52e730190fc',
 '0x3a925503970d40d36d2329e3846e09fcfc9b6acb',
 '0xd18748b9839b0081a867a1a871d5b562b2ec9884',
 '0x9d640080af7c81911d87632a7d09cc4ab6b133ac',
 '0x231b7589426ffe1b75405526fc32ac09d44364c4',
 '0xd4405f0704621dbe9d4dea60e128e0c3b26bddbd',
 '0x4bb0925fa50da9b4c8936869433b48e78ccc5c13',
 '0xebd17511f46a877199ff08f0ea4f119c9b4aea50',
 '0x7f75f0aed3c4158b91ad3641afa6e8d035311862',
 '0x64b91b92240bc1901855dd55ae632addb650d089',
 '0x163d38e33b16697e69249665daadb50be81e7f72',
 '0x0cfb06414c6d9790bc661230dba0b24060808bf4',
 '0x004375dff511095cc5a197a54140a24efef3a416',
 '0x5fa9569b0ed6aa01e234468e6a15b77988b950df',
 '0x51bf9908dac4a283d017edc071d954ec3a4b1376',
 '0xbbf933c1af0e9798615099a37a17cafc6da87732',
 '0xf0d1109e723cb06e400e2e57d0b6c7c32bedf61a',
 '0xde6faedbcae38eec6d33ad61473a04a6dd7f6e28',
 '0xb75767451fc277211a480ff3712eac6b0fa23b13',
 '0xf6887ad6f3b342840dbea138e8003f2ca14f4500',
 '0xea38ac8ce7756be643341bff2c34c8d975215506',
 '0xea7952fac7ff6e997d895c1566599b86b91444c0',
 '0x4f7be8cdd826f20d03a70f8a47b63b2dbd4092fd',
 '0x919b75ff6c4e4e89528641cf13e1d7f45de3ac03',
 '0xeacd8a344c16b5724819c7a1c253da07f2ca50a4',
 '0x426c02d6ce5fde4ae7556e7345f8101f64807a7c',
 '0x9ccebb68580a6c555a0c01592f1b95050f3d13fe',
 '0x5b8fb733f1a427e68533db48b7210d1548ee1dcd',
 '0x3e436c4908c64c93d80ca6a2694b53bd970c4e33',
 '0x3b44f35fd81040e4d0cdccfe77714512feb1da4d',
 '0xe266486afe9763c93530893e90264144c3a4023d',
 '0xb6694aeaaf22abba85c481b67551da9e073714ea',
 '0xa219aad740f88470c59a3ddbaf6ded12e7c3fb76',
 '0x9f624b25991b99d7b14d6740a9d581dd77980808',
 '0x88fcdd9cd5ee7e53ba452c45f7f704041df0b042',
 '0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d',
 '0x5f094191ddad05c5b6ddfa992a4f49d8d679290d',
 '0xde93684627d4e34d6ef96adf5bcabf10bbd8dd81',
 '0xb4c471205e8c357e10bbecda80b6aefe9351f01b',
 '0x4d5ebb22982ffeccb7b3e42a624555cb313285f0',
 '0x689d2ee154633ae19d61a8570c09248dcbd0ed04',
 '0x873823930b78e1c1c3d9674b5cb107dd2d211bf0',
 '0x97e58528d0c7bf0d0089f52be1dec3565d84c46a',
 '0xb405652045933afdcdc6fcbbfb4d0ecaf33c67f0',
 '0x48356de0473fa6993ec68a1c5e04da2896becf7c',
 '0x1b87fde6af5396165fdadf7f532784622a824abf',
 '0xeb4b2b5e0eae7a0eadd0673ef8c3c830f8762f28',
 '0x3f61171b1297f6e512c750c172ba86026ab0e820',
 '0xb5b51148a602afb716e241273adbce5558540293',
 '0x576cea6d4461fcb3a9d43e922c9b54c0f791599a',
 '0x2b9f8fe8ffc437a9008bb3097066f02b0a1c52ec',
 '0x6b4de908a8329c95c63414375612fe52ffb8792e',
 '0x63cd4c240722b2fbb60e0d1697be697450625f50',
 '0x1812ef69e1753f908229df40d304a5fbdcd52dd3',
 '0x31cf0327782a22958e7361f75ed86a7b69eb0aa7',
 '0xd14782a6f2512495584a252ffca49b17486a08d3',
 '0x4085cafb2c519e6bf69ad410d91dbb6139948366',
 '0x81a10a45143d7a6b7e1a0e63761b4fba311bb661',
 '0x88ff79eb2bc5850f27315415da8685282c7610f9',
 '0x5522c339d35a6c3e5f328c8746bdf88f599dab83',
 '0xe618785149a1ee7d3042e304e2f899f7a4616b7d',
 '0xbce060d3d8e6ffc3b874dba775a0111609b87d90',
 '0xacbf3c973eeec20c573ec930ce31198afb4ad0d5',
 '0xb76bcf6b873cbd749854a90d477e6fe1fcbe86a6',
 '0x77d5e268b26874f849425ea13755743b271a3050',
 '0xd16d38071f315ac0860753e43c2ee022c8e09b1b',
 '0x507d84fe072fe62a5f2e1f917be8cc58bdc53ef8',
 '0x4b8852e4747b8a7d4caf0440e4e3397032a6723d',
 '0xbdd344f29c737c6d93c956a112c25b44ee178cba',
 '0xea02e0f8f470de1107cb336f827c4192117574bb',
 '0x0ddcda72833e8cbc4f94c24bc7a1baf14853e76e',
 '0x819de42d3ab832eaf7111a222a8a5a7419f13b48',
 '0x514906fc121c7878424a5c928cad1852cc545892',
 '0xf8f571d2ace4c5c118933a3f8a8654d51ba455dd',
 '0x0dbd5d63e04aadee8641b04829d125e3943c6b19',
 '0x1fead6b98371793a0897b3d1a9402e4bb9906775',
 '0xe700b18c24dc447b8b766c76f813c7a6840bd73b',
 '0xb039f945deca4b4ec44c28083f91f37e6d2a27a6',
 '0x9e251daeb17981477509779612dc2ffa8075aa8e',
 '0x66e32d1b776a43935ed20e8dc39a27a140d31c8f',
 '0x66e33d2605c5fb25ebb7cd7528e7997b0afa55e8',
 '0x7b034bba28ec5f3a4f844ca1ee7178af06f7d0e9',
 '0x3ef9995a4ccce7402d02d92216b394c0c3cc1b1b',
 '0x1a57ec5459928389fbb5612ff2a5e0b534fd9e2e',
 '0xb43542e6b7c209f7da4b579792f67a1844546ce8',
 '0x9b0d4323c32b5d9a87409b435adf7e23cc5f835c',
 '0x5ed0dde7441efcd728e31cfd055c4c7efe20a57b',
 '0x76911e11fddb742d75b83c9e1f611f48f19234e4',
 '0x2702bd7268793b5e1c7ee1ac2d9cae2ae2ecfe55',
 '0xf8f2ac05e7056fd83668bf395ba78a363a3d00b5',
 '0x35fe623f3cb6e81091738f7f659fddd72e66980f',
 '0xf1ee7348e45ef0e838c705c2eece6d414d4aab3f',
 '0xd6e1720b9a3bce9925e29f6b05204748097d287b',
 '0x3e78f2e7dade07ea685f8612f00477fd97162f1e',
 '0x197e99bd87f98dfde461afe3f706de36c9635a5d',
 '0xa8ef4df1577be1a1b4e884eb67aed494f39dc36d',
 '0xed9ca2a60d683c9ba04fd2bc043aad76a0afc59f',
 '0x02ecc701259acd166f67e912e75ce169c7bc8fdf',
 '0xc4f0ff5e1ddc6752a13bf0f59ad2beb2fc25f175',
 '0xc2bfb44db07576a3736dfa554750a81ff67cbde2',
 '0xf01944044f064049f1d5663d5cc7239b82049228',
 '0xb99c23a9a444ebeb0ce4a67f27dab8d4826b1108',
 '0xe1ab0442520d5b8bcf09317ad7e0c5bad5824fcd',
 '0x4b8fc31bbef97440836e7316a7c807aa90644e3b',
 '0xd91c72da93288f00385e3c2ead75623ebedf9272',
 '0xe6bbc807cb59b5f280b835135d01d077657d4550',
 '0xa5bd4a268edf1d808bc0cd0591613f998a2193ad',
 '0x436b56d0c933a4660594fc774a3e904c2c21b22a',
 '0xcd6f65a972551ffac9b52fa2d4a561d9b7ab4741',
 '0x35a1f588dbd86446a4eb8133ccc96e3db0e65be4',
 '0x8a46eb07ed7c3d6f883990d32aad0f8231901a64',
 '0xd3ff34fe9692c563bafcb58c9a085c8c89c2f180',
 '0xd02ac36f729b867174e7361d5f272c542c71ad73',
 '0x78bf78ac09c891a6538b93cb90b0b2496a0746f8',
 '0x7f0ad87b99ba16e6e651120c2e230cf6928c3d15',
 '0x8731bbfd79ec1a0d363d5ec21c88979ae630c805',
 '0xb4d0d9df2738abe81b87b66c80851292492d1404',
 '0xca96653bebe79a3ba8bf3c4e2198f20fb5bd047f',
 '0xac9fbdbe486f8023606b932a747bc476011b5071',
 '0x9d180c1baf1de69e9b922625284e1a2211a6ebea',
 '0xb63c2f390803119a98ac63a371bcc657e366df0f',
 '0xfc0dd985f6de9c2322ebd97c3422b0857c4d78c7',
 '0xd4981ed8892cacc135192385de05b19684231e74',
 '0x169cd8eaa348d3dbbe542302f651c53a1e43c736',
 '0xa02b22703bbd60ca2145b64ea55dd2cdd0bc2a65',
 '0xa7b0d77e322f7643f6eb7c8bb4ff83e49c6bdee8',
 '0x524ae259179691bd2d727cada0e1aa4e365824e9',
 '0xd42e338a6fcf2e343afffd4eca59e8eca64bad39',
 '0xedf1fa564a91a5664f172470c47450af17724757',
 '0x524847c615639e76fe7d0fe0b16be8c4eac9cf3c',
 '0xc992a50169f6075d52013118355c633bf92ae853',
 '0x4c750909db4a83179d7aee1ecec497309075a499',
 '0xbbb9bf440d0f686487925fef3b0a0f9aa67753f6',
 '0xebfb684dd2b01e698ca6c14f10e4f289934a54d6',
 '0xf00e80f0de9aea0b33aa229a4014572777e422ee',
 '0x20b2ef197302be35c4c08f466a646e5da75a86e3',
 '0x8014851acaa37e277e7369c5a850f29a152b1169',
 '0x763bb98b2f504882ef3a2eb62b4c8003b94c59cc',
 '0xbb12d6020e80e9f329f72f9d2209807d2e49d3f8',
 '0x5e072bbdcb4ed704763d8c9208179e05a7ca5943',
 '0x5ac13261c181a9c3938bfe1b649e65d10f98566b',
 '0x0de0fa91b6dbab8c8503aaa2d1dfa91a192cb149',
 '0x880715a29b569cb44295a9fa05c8dd1b4fff33b4',
 '0x88c095c8ba2c7a1353cf3d21e692c5d4d0f90793',
 '0xabd9c284116b2e757e3d4f6e36c5050aead24e0c',
 '0xdc93e9f3e9d895323c83480d97410ab1246f7eb4',
 '0x4f1d67645be3caffc3d1346fb6c69b631db11858',
 '0xe1ae2769d69e823567157ad40cf89466cd615ed1',
 '0x4a5cf9ecc6fdd4750df92a33ced79d477d9298c8',
 '0x853a43cc651e8b25e753eed19dde68c6a6cab254',
 '0x3eed0af1c5f350c6571525d9e3eeea7d2608af81',
 '0xdf6957dea7dbdf660cc440160712ea6163f22e8f',
 '0x773dd321873fe70553acc295b1b49a104d968cc8',
 '0x569b80f81e7d3514991780b4df32b533dda798a1',
 '0xeb5455590bb2bf67d54d7d3affaa1930c5132b14',
 '0x611805ce4071c096f5b44557b45685802d8a46fe',
 '0xd9999f7f26af362207807be9d78daf2c9caacda2',
 '0x5e4a92625a337707ba362abff463bccc5b896d21',
 '0xa0abda1f980e03d7eadb78aed8fc1f2dd0fe83dd',
 '0xc628da629c615909c961b1c69c3bcac0c3fad8a9',
 '0x7585d62f7a00cfb3870fe0a563002006af189e0f',
 '0x278fd401240b47692dd483c050377fa3ac6f6ba4',
 '0x4e26311405906586742de36ad7db4ca2b6dab842',
 '0x46231ddd9b610140f8ceed7eb29bc0dcfc6cc1c0']

In [48]:
for contract_id in contracts_list[125:]:
    extract_history(contract_id)
    

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.79s/it]

list index out of range



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

list index out of range


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.83it/s]

...

list index out of range



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

list index out of range


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.34it/s]

list index out of range


In [53]:
len(contracts_list)

244